In [1]:
import os
import json
import requests
from datetime import datetime
import pandas as pd
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

/Users/vasilishatzopoulos/miniforge3/envs/ai/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


True

In [2]:
def authenticate():
    
    url = "https://auth.emsicloud.com/connect/token"

    CLIENT_ID = os.getenv("CLIENT_ID")
    CLIENT_SECRET = os.getenv("CLIENT_SECRET")
    SCOPE = os.getenv("SCOPE")
    payload = f"""client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&grant_type=client_credentials&scope={SCOPE}"""
    print(payload)
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}

    response = requests.request("POST", url, data=payload, headers=headers)

    return json.loads(response.text)["access_token"]

In [3]:
def get_skills(acess_token):

    url = url = "https://emsiservices.com/skills/versions/latest/skills"
    headers = {'Authorization': f"""Bearer {access_token}"""}
    response = requests.request("GET", url, headers=headers)
    path = os.path.join(os.getenv("DATA"),"Lightcast/Skills",f"""skills_{str(datetime.today())[:10]}.json""")
    with open(path, 'w') as file:
        json.dump(response.text, file, indent=4)
    
    


In [4]:
def get_jobs(acess_token):

    url = "https://emsiservices.com/titles/versions/latest/titles"
    headers = {'Authorization': f"""Bearer {access_token}"""}
    response = requests.request("GET", url, headers=headers)
    path = os.path.join(os.getenv("DATA"),"Lightcast/Jobs",f"""jobs_{str(datetime.today())[:10]}.json""")
    with open(path, 'w') as file:
        json.dump(response.text, file, indent=4)


In [45]:
def extract_skills(access_token):

    #https://docs.lightcast.dev/apis/skills#versions-version-extract
    url = "https://emsiservices.com/skills/versions/latest/extract"
    headers = {'Authorization': f"""Bearer {access_token}""",
               'Content-Type': "application/json"}
    text = "The Data Scientist II provides technical leadership across the overall Analytics function which may have an enterprise mandate. Role generally provides deep technical knowledge and expertise in client interactions to explain complex data analysis related material.\n     Depth & Scope:\n    \n      Generally accountable for a significant business management area that typically has enterprise wide impact or accountability\n      Enterprise or functional expert, requiring broad managerial and deep specialized knowledge at the enterprise, business, regulatory and industry levels\n      Undertakes and completes a variety of complex initiatives requiring seasoned specialist knowledge and/or the integration of cross functional processes\n      Position typically deals with senior/executive management\n      Focuses on longer-range planning for functional area (e.g. 12 months or greater)\n      May manage and prioritize multiple projects at a given time\n    \n    \n  \n  \n   Job Requirements \n   \n    Education & Experience: \n    \n     Undergraduate degree or technical certificate\n      7+ years relevant experience\n      Proficient PC skills including a strong competency in MS Excel and PowerPoint, and a variety of PC based analytical and reporting software packages\n      Advanced experience in the knowledge, utilization and adoption of the latest tools (i.e. machine learning) and technologies (i.e. Hadoop) in big data and advanced analytics\n      Strong analytical and program solving skills are required to interpret data and draw conclusions\n      Excellent written and verbal communications skills\n    \n    \n      Must be eligible for employment under regulatory standards applicable to the position.\n    \n    \n  \n  \n   Company Overview \n   \n    Our Values\n     At TD, we're guided by our purpose to enrich the lives of our customers, communities and colleagues, and share a set of values that shape our culture and behaviors. In exchange for how our colleagues show up to help TD succeed, we are committed to delivering a colleague experience grounded in Impact, Growth and a Culture of Care. No matter where you work across TD, we empower you to make an impact at work and in your community, explore and grow your career, and be part of our caring and inclusive culture.\n     \n     Making Your Well-being a Priority\n     A caring and supportive culture that promotes colleague well-being is core to who we are. At TD, we focus on total well-being with extensive programs to help colleagues assess, manage, and improve their well-being across four core pillars — physical, financial, social and mental/emotional. In addition, we champion a safe and inclusive work environment, so colleagues feel a sense of belonging and feel supported in their personal and professional growth. Through our focus on well-being, we know that we can help our colleagues thrive, contribute to our culture of care, and support better business outcomes, because when colleagues feel their best, they're more likely to do their best.\n     \n     Our Total Rewards Package\n     Our Total Rewards package reflects the investments we make in our colleagues to help them and their families achieve their financial, physical and mental well-being goals. Total Rewards at TD includes base salary and variable compensation/incentive awards (e.g., eligibility for cash and/or equity incentive awards, generally through participation in an incentive plan) and several other key plans such as health and well-being benefits, savings and retirement programs, paid time off (including Vacation PTO, Flex PTO, and Holiday PTO), banking benefits and discounts, career development, and reward and recognition.\n     \n     How We Work\n     At TD, we believe in-person connections fuel collaboration and collective creativity. Our workplace experience empowers colleagues to do great work side-by-side at TD locations, while offering flexibility to work remotely where it makes sense for the work and team. Our teams work in one of three workplace models: hybrid, onsite and primarily remote. Wherever our colleagues are working, they’ll always have access to the TD community to experience our culture of care.\n     \n     Who We Are\n     TD is one of the world's leading global financial institutions and is the fifth largest bank in North America by branches/stores. Every day, we deliver legendary customer experiences to over 27 million households and businesses in Canada, the United States and "
    payload = {"text":text,'confidenceThreshold': 0.6}
    #payload = "{ \"text\": \"... Great candidates also have\\n\\n Experience with a particular JS MV* framework (we happen to use React)\\n Experience working with databases\\n Experience with AWS\\n Familiarity with microservice architecture\\n Familiarity with modern CSS practices, e.g. LESS, SASS, CSS-in-JS ...\", \"confidenceThreshold\": 0.6 }"

    response = requests.request("POST", url, data=json.dumps(payload), headers=headers)
    print(response.text)
    
    data  = json.loads(response.text) 
    return [d['skill']['name'] for d in data["data"]], response.headers

In [6]:
def read_Lightcast_catalogue(data_path):
    with open(data_path, 'r') as file:
        data = json.load(file)
    data = json.loads(data)
    return data
    

In [7]:
def write_catalogues_to_csv():
    
    data_path = os.getenv("DATA_SKILLS")
    skills = read_Lightcast_catalogue(data_path)["data"]
    skill_catalogue = [datum['name'] for datum in skills]
    skill_catalogue = pd.DataFrame(skill_catalogue)
    skill_catalogue.columns = ['skill']
    file_path = os.path.join(os.getenv("DATA"),"Lightcast/Skills","skill_catalogue.csv")
    print(file_path)
    skill_catalogue.to_csv(file_path,index=None)

    data_path = os.getenv("DATA_JOBS")
    jobs = read_Lightcast_catalogue(data_path)["data"]
    jobs_catalogue = [datum['name'] for datum in jobs]
    jobs_catalogue = pd.DataFrame(jobs_catalogue)
    jobs_catalogue.columns = ['job']
    file_path = os.path.join(os.getenv("DATA"),"Lightcast/Jobs","job_catalogue.csv")
    print(file_path)
    jobs_catalogue.to_csv(file_path,index=None)

In [ ]:
access_token = authenticate()
get_skills(access_token)
get_jobs(access_token)
write_catalogues_to_csv()


In [46]:
access_token = authenticate()


client_id=7sn7t35qti9fmetr&client_secret=pJUU4IGA&grant_type=client_credentials&scope=emsi_open


In [47]:
data,headers = extract_skills(access_token)

{"attributions": [{"name": "Wikipedia", "text": "Wikipedia extracts are distributed under the CC BY-SA license (https://creativecommons.org/licenses/by-sa/3.0/)"}], "data": [{"confidence": 1.0, "skill": {"description": "Apache Hadoop is a collection of open-source software utilities that facilitates using a network of many computers to solve problems involving massive amounts of data and computation. It provides a software framework for distributed storage and processing of big data using the MapReduce programming model. Hadoop was originally designed for computer clusters built from commodity hardware, which is still the common use. It has since also found use on clusters of higher-end hardware. All the modules in Hadoop are designed with a fundamental assumption that hardware failures are common occurrences and should be automatically handled by the framework.", "descriptionSource": "https://en.wikipedia.org/wiki/Apache_Hadoop", "id": "KS124KS624R3L2W8RKM6", "infoUrl": "https://light